In [ ]:
# 1. Install dependencies (if not already installed)
!pip install langchain langchain-openai langchain-community chromadb

# 2. Load CSV data
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.documents import Document

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.4 MB/s eta 0:00:0

In [ ]:
# Load the CSV file
df = pd.read_csv("bank_faqs.csv")  # Update path if needed


In [ ]:
df

,question,answer
0,How can I open a savings account?,You can open a savings account by visiting you...
1,What documents are required for a personal loan?,"You need to provide ID proof, address proof, i..."
2,How can I check my account balance?,You can check your account balance via mobile ...
3,What is the minimum balance requirement?,The minimum balance requirement depends on the...
4,How to block my lost debit card?,"To block your lost debit card, call our 24x7 c..."
5,Can I get a home loan pre-approval?,"Yes, you can apply for a home loan pre-approva..."
6,What is the interest rate on fixed deposits?,Our fixed deposit interest rates range from 3....
7,How do I update my mobile number?,You can update your mobile number by visiting ...
8,How can I get a new cheque book?,You can request a new cheque book through mobi...
9,Is there a penalty for late EMI payment?,"Yes, a penalty is charged for late EMI payment..."


In [ ]:
# Convert each row to a LangChain Document (for embedding)
docs = [
    Document(page_content=f"Q: {row['question']} A: {row['answer']}")
    for _, row in df.iterrows()
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

In [ ]:

print("Total chunks created:", len(split_docs))

Total chunks created: 20


In [ ]:
# 4. Set up OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = " "  # Replace with your real key

In [ ]:
!pip install langchain_chroma

In [ ]:
# 5. Embed and store in Chroma vector store
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(split_docs, OpenAIEmbeddings())

In [ ]:
# 6. Set up retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
# 7. Create the LLM and prompt
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [ ]:
llm = OpenAI(temperature=0.3, max_tokens=500)

In [ ]:
system_prompt = (
    "You are an expert banking assistant designed to provide clear, concise, and accurate answers "
    "to customer frequently asked questions based on the provided context. "
    "Follow these guidelines:\n\n"
    "1.  **Strictly use the provided context**: Only use the information given in the '{context}' section to formulate your answers.\n"
    "2.  **Direct and concise answers**: Get straight to the point. Avoid conversational filler.\n"
    "3.  **Maintain a helpful and professional tone**: Your language should be polite and easy to understand.\n"
    "4.  **Address the user's specific question**: Ensure your answer directly addresses the query posed in the '{input}' section.\n"
    "5.  **Handle out-of-scope questions**: If the provided context does not contain the answer to the question, respond with:\n"
    "    'I'm not sure about that based on the information I have. Please contact customer support for further assistance.'\n"
    "6.  **Format for clarity**: Present the answer clearly, perhaps using bullet points if applicable, but only if the context supports it.\n"
    "7.  **Avoid making assumptions**: Do not infer or add information not present in the context.\n"
    "8.  **Prioritize accuracy**: Ensure the information you provide is a faithful representation of the retrieved context.\n\n"
    "Retrieved information:\n{context}"
)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [ ]:
# 8. Build the RAG pipeline
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# 9. Ask a question
query = "How can I open a savings account?"
response = rag_chain.invoke({"input": query})
print("Q:", query)
print("A:", response["answer"])

Q: How can I open a savings account?
A: 
System: To open a savings account, you can either visit your nearest branch with the required documents or apply online through our website.


In [ ]:
!pip install -qU gradio

In [ ]:
import gradio as gr

# Gradio chatbot function
def chat_with_rag(user_question):
    response = rag_chain.invoke({"input": user_question})
    return response["answer"]

# Launch Gradio interface
gr.Interface(
    fn=chat_with_rag,
    inputs=gr.Textbox(label="Ask a banking question"),
    outputs=gr.Textbox(label="Answer"),
    title="Bank FAQ Chatbot",
    description="Ask questions about banking services powered by RAG and OpenAI."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7f50ec9aaa0bc64786.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def bank_rag_assistant_lc(message, history):
    out = rag_chain.invoke({"input": message})
    # LangChain retrieval chain returns a dict with "answer"
    return out.get("answer", "").strip() or "Sorry, I couldn't generate an answer."

interface = gr.ChatInterface(
    fn=bank_rag_assistant_lc,
    title="Bank FAQ Chatbot",
    description="RAG over your FAQs using LangChain + Chroma",
)

interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e75c2798ba2cc46d24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
